In [1]:
import pandas as pd

# Load the full dataset if not already loaded
df = pd.read_csv('Data1_WIS.csv')
# Create sender-recipient mapping for names
senders = df[['SENDER_DIM_ACCOUNT_KEY', 'SENDER_NAME']].rename(columns={'SENDER_DIM_ACCOUNT_KEY': 'node','SENDER_NAME': 'Name'})
        
recipients = df[['RECEPIENT_DIM_ACCOUNT_KEY', 'RECEPIENT_NAME']].rename(columns={'RECEPIENT_DIM_ACCOUNT_KEY': 'node','RECEPIENT_NAME': 'Name'})

user_info = pd.concat([senders, recipients]).drop_duplicates(subset='node')
print(user_info)

                                   node                 Name
0      89eac5a629485d0c750365842df60e47         ABBY Madonia
43     ce301530ec2181dfe08f6424f3b7ea4c         ADDIE Hirsch
58     d17dc2729be542b3f8d4abcdbb74a6ae    ADDISON ARMSTRONG
70     5b7bd60d536752728d89d6c3d5210131    ADDISON Sacharski
73     f2efa70d1b230fdcdbbca9f515e0bb27          ADDY Piette
...                                 ...                  ...
78127  e5ab58446c707059cd142dc0b810dc94  Peter Vanden Heuvel
78128  ce2c5fa4ca280500e401cd108c56efad       Jenel Tompkins
78129  5e9ffd32257ac5a219e6835d197c9e6e       Justin Gibbons
78130  0634093a501a2db88c4c33eefec08603           Marcel Kas
78131  59aadbb72c169a1575d7ab7af4b05ff7    Brendan Grabowski

[63486 rows x 2 columns]


In [2]:
# --- Load data ---
df = pd.read_csv('Data1_WIS.csv')

# --- Rename recipient column if needed ---
df.rename(columns={'RECEPIENT_DIM_ACCOUNT_KEY': 'RECIPIENT_DIM_ACCOUNT_KEY'}, inplace=True)

# Clean and convert comma-separated strings into lists
df['ACTIVITIES'] = df['ACTIVITIES'].fillna('').apply(lambda x: [a.strip() for a in x.split(',')] if x else [])

# Now explode to one row per activity
df_exploded = df.explode('ACTIVITIES')

# Group by sender and activity, then sum transfers
activity_transfer_counts = (
    df_exploded
    .groupby(['SENDER_DIM_ACCOUNT_KEY', 'ACTIVITIES'])['TOTAL_TRANSFERS']
    .sum()
    .unstack(fill_value=0)
    .add_suffix('_transfers')
)

# Step 4: Core sender summary as before
sender_summary = df.groupby('SENDER_DIM_ACCOUNT_KEY').agg({
    'SENDER_NAME': 'first',
    'PREFERRED_EMAIL': 'first',
    'TICKET_BILLING_ZIP': 'first',
    'TOTAL_TRANSFERS': 'sum',
    'TRANSFERS_2020': 'sum',
    'TRANSFERS_2021': 'sum',
    'TRANSFERS_2022': 'sum',
    'TRANSFERS_2023': 'sum',
    'TRANSFERS_2024': 'sum',
    'RECIPIENT_DIM_ACCOUNT_KEY': pd.Series.nunique,
}).rename(columns={'RECIPIENT_DIM_ACCOUNT_KEY': 'unique_recipients'})
    
# Step 5: Merge in activity columns
sender_summary = sender_summary.join(activity_transfer_counts, how='left').fillna(0)

# 4. Reset index
sender_summary.reset_index(inplace=True)
    
# 5. Save to CSV
sender_summary.to_csv("sender_level_consolidated_WIS.csv", index=False)
print(sender_summary)

                SENDER_DIM_ACCOUNT_KEY      SENDER_NAME  \
0     0000852685cb25ecf38e0420f217356e      Miriam Sham   
1     000d2c9c6e10fbf68829c8e32e2825da   Meghan McGuire   
2     001c3954e549a12b4c87db02dbd07584  Charles Stathas   
3     00294a0e7108ddb6d35a706328c0b277     Jeff Abraham   
4     002b0d73b366ad618a44fca5325d46b5   Erin McQuillan   
...                                ...              ...   
9994  ffe57e0d564164028790aa2f10282257   Henry Andersen   
9995  ffeb1b975390f5dc82137449cf2cbdb3    Gracie Becker   
9996  ffebee2d602d3e1288a1373c119a923d    Howard Paster   
9997  fff2a8a5ff8fe41af77f1b2c2ec0ee11       Jason Adix   
9998  fff91ada6ab3f75c85a0ff544ccc6622    Bennett Unger   

               PREFERRED_EMAIL TICKET_BILLING_ZIP  TOTAL_TRANSFERS  \
0              msham3@wisc.edu              53022                5   
1           mfmcguire@wisc.edu              55347                3   
2     charlesstathas@gmail.com              53718                5   
3     Jkabr

In [5]:
import pandas as pd
import networkx as nx
from collections import Counter
import community.community_louvain as community_louvain

# --- Create directed graph ---
G_full = nx.from_pandas_edgelist(
    df, 
    source='SENDER_DIM_ACCOUNT_KEY', 
    target='RECIPIENT_DIM_ACCOUNT_KEY', 
    edge_attr='TOTAL_TRANSFERS', 
    create_using=nx.DiGraph()
)

# --- Louvain community detection (on undirected version) ---
partition = community_louvain.best_partition(G_full.to_undirected())
nx.set_node_attributes(G_full, partition, 'community')

# --- Out-degree ---
out_deg = dict(G_full.out_degree(weight='TOTAL_TRANSFERS'))
nx.set_node_attributes(G_full, out_deg, 'out_degree')

# --- Centrality Measures ---
deg_centrality = nx.degree_centrality(G_full)
btwn_centrality = nx.betweenness_centrality(G_full)
try:
    eigen_centrality = nx.eigenvector_centrality(G_full.to_undirected(), max_iter=500)
except nx.PowerIterationFailedConvergence:
    eigen_centrality = {n: None for n in G_full.nodes()}
    print("⚠️ Eigenvector centrality did not converge.")

# --- Add all node attributes to DataFrame ---
centrality_df = pd.DataFrame({
    'node': list(G_full.nodes()),
    'community': [partition[n] for n in G_full.nodes()],
    'out_degree': [out_deg.get(n, 0) for n in G_full.nodes()],
    'degree_centrality': [deg_centrality.get(n, 0) for n in G_full.nodes()],
    'betweenness': [btwn_centrality.get(n, 0) for n in G_full.nodes()],
    'eigenvector': [eigen_centrality.get(n, None) for n in G_full.nodes()]
})

# --- Attach sender metadata ---
sender_info = df[['SENDER_DIM_ACCOUNT_KEY', 'SENDER_NAME', 'PREFERRED_EMAIL', 'TICKET_BILLING_ZIP']] \
                .drop_duplicates(subset='SENDER_DIM_ACCOUNT_KEY') \
                .rename(columns={
                    'SENDER_DIM_ACCOUNT_KEY': 'node',
                    'SENDER_NAME': 'name',
                    'PREFERRED_EMAIL': 'email',
                    'TICKET_BILLING_ZIP': 'zip'
                })

# --- Merge metadata ---
final_df = pd.merge(centrality_df, sender_info, on='node', how='left')

# --- Save to CSV ---
final_df.to_csv('sender_global_centrality_WIS.csv', index=False)
print("✅ Saved to sender_global_centrality.csv")


✅ Saved to sender_global_centrality.csv


In [3]:
import pandas as pd

# --- Load all datasets ---
centrality_df = pd.read_csv("sender_global_centrality_WIS.csv")               # node = sender_id
consolidated_df = pd.read_csv("sender_level_consolidated_WIS.csv")           # SENDER_DIM_ACCOUNT_KEY
ctr_df = pd.read_csv("WIS_clickthrough.csv")                          # must contain SENDER_DIM_ACCOUNT_KEY
web_df = pd.read_csv("WIS_webactivity.csv")                                 # must contain SENDER_DIM_ACCOUNT_KEY

don_df = pd.read_csv("all_donations_wis.csv")
don_df = don_df.rename(columns={'ID': 'SENDER_DIM_ACCOUNT_KEY'})

# --- Standardize sender ID column name ---
centrality_df.rename(columns={'node': 'SENDER_DIM_ACCOUNT_KEY'}, inplace=True)

# --- Merge all ---
merged_df = consolidated_df.merge(centrality_df, on='SENDER_DIM_ACCOUNT_KEY', how='left')
merged_df = merged_df.merge(ctr_df, on='SENDER_DIM_ACCOUNT_KEY', how='left')
merged_df = merged_df.merge(web_df, on='SENDER_DIM_ACCOUNT_KEY', how='left')
merged_df = merged_df.merge(don_df, on='SENDER_DIM_ACCOUNT_KEY', how='left')

# --- Save final merged dataset ---
merged_df.to_csv("final_model_dataset_WIS.csv", index=False)
print("✅ Final dataset saved as final_model_dataset.csv")

✅ Final dataset saved as final_model_dataset.csv
